In [2]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re

In [3]:
#change path
path="..\\UB_keystroke_dataset\\S0"

In [4]:
def ReadListOfTextFilesFromDirectoryWalk(path):
    """Read all text files from given directory and its sudirectories and return list of text files path"""
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r, file))
    return files



In [5]:
files=ReadListOfTextFilesFromDirectoryWalk(path)
len(files)
print(files[0:1])

[]


In [6]:
def CreateListOfTuplesFromFile(files):
#Reading file and create tuple 
    items = []
    for file in files:
        filename=file.split('\\')[-1].split('.')[0]
        f=open(file, "r")
        for line in f:
            words= line.rstrip('\n').split(sep=" ")
            items.append((words[0],words[1],words[2],filename))
    return items


In [7]:
items=CreateListOfTuplesFromFile(files[0:2])
items

[]

In [8]:
#Creating dataframes
def CreateDataFrame(items,columns):
    df = pd.DataFrame.from_records(items, columns=['Key', 'EventType','Time','User'])
    return df

In [9]:
df=CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
print (df.shape)
#
df.head(10)

(0, 4)


,Key,EventType,Time,User


In [10]:
def ParseAlphabetsKeys(df):
    """Return only alphabets records"""
    dfAlphabets=df[df["Key"].str.match('^.*[A-Z]$')]
    return dfAlphabets

In [11]:
temp=ParseAlphabetsKeys(df)
temp.groupby("Key").size()/2

Series([], dtype: float64)

In [12]:
temp

,Key,EventType,Time,User


In [13]:
def GetTimeDifferenceofKeyDownDown(data):
    prev=0
    rows=[]
    for index,row in data.iterrows():
        if prev==0 and row[1]=="KeyDown":
            prevRow=row
            prev=1
        elif row[1]=="KeyDown":
            rows.append([ prevRow[0], row[0], int(row[2])- int(prevRow[2]),row[3]])
            prevRow=row
    return rows

In [14]:
keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=temp)
keyDownDownTimeDifference


[]

In [15]:
def GetDictionaryOfFeatureVectors(keyDownDownTimeDifference):
    """
    Take List of arrays and return dictionary of 26*26 vector for each user
    """
    dictFV=dict()
    for xi in keyDownDownTimeDifference:
        if xi[3] not in dictFV:
            dictFV[xi[3]]=np.zeros(26*26,dtype=object)
            
        index=(ord(xi[0])-65)*26+np.absolute(ord(xi[1])-65)
        if dictFV[xi[3]][index]==0:
            dictFV[xi[3]][index]=[]
        dictFV[xi[3]][index].append(xi[2])
    return dictFV


In [16]:
FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
usersFV=pd.DataFrame.from_dict(FeatureVectors)
GMMData=[]
for user in usersFV.columns:
    maxLength=0
    for userF in usersFV[user]:
        if userF==0:
            userF=[]
        featureLenght=len(userF)
        if featureLenght > maxLength:
            maxLength=featureLenght
    userDF=pd.DataFrame(index=range(0,676))
    userData=np.zeros((maxLength,676))
    row=0
    for userF in usersFV[user]:
        column=0
        if userF==0:
            userF=[]
        availableValuesCount=len(userF)
        sum=0
        for value in userF:
            userData[column][row]=value
            column=column+1
            sum=sum+value
        if availableValuesCount==0:
            mean=0
        else:
            mean=sum/availableValuesCount
        gussianValues=np.random.normal(mean,3,maxLength-availableValuesCount)
        for value in gussianValues:
            
            userData[column][row]=value
            column=column+1
        
        row=row+1
    print(userData.shape)
    if len(GMMData)==0:
        GMMData=userData
    else:
        GMMData=np.append(GMMData,userData,axis=0)

print(GMMData.shape)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
x1='Z'
x2='Z'
(ord(x1)-65)*26+np.absolute(ord(x2)-65)


In [ ]:
x1='Z'
x2='Z'
(ord(x1)-65)*26+np.absolute(ord(x2)-65)


In [ ]:
def GetGMMFormatDataFromDataFrame(usersFV):
    GMMData=[]
    for user in usersFV.columns:
        maxLength=0
        for userF in usersFV[user]:
            if userF==0:
                userF=[]
            featureLenght=len(userF)
            if featureLenght > maxLength:
                maxLength=featureLenght
        userDF=pd.DataFrame(index=range(0,676))
        userData=np.zeros((maxLength,676))
        row=0
        for userF in usersFV[user]:
            column=0
            if userF==0:
                userF=[]
            availableValuesCount=len(userF)
            sum=0
            for value in userF:
                userData[column][row]=value
                column=column+1
                sum=sum+value
            if availableValuesCount==0:
                mean=0
            else:
                mean=sum/availableValuesCount
            gussianValues=np.random.normal(mean,3,maxLength-availableValuesCount)
            for value in gussianValues:
            
                userData[column][row]=value
                column=column+1
        
            row=row+1
        print(userData.shape)
        if len(GMMData)==0:
            GMMData=userData
        else:
            GMMData=np.append(GMMData,userData,axis=0)
    return GMMData


In [ ]:
data= GetGMMFormatDataFromDataFrame(usersFV)

In [ ]:
def GetAverageFeatureVectors(FeatureVectors):
    
    averageFV= np.zeros(26*26,dtype=object)
    index=0
    for xi in FeatureVectors:
        tempSum=np.array(xi).sum()
        if type(xi)== list:
            averageFV[index]=tempSum/len(xi)
        index=index+1
    return averageFV

In [ ]:
for user in FeatureVectors:
    GetAverageFeatureVectors(FeatureVectors[user])

In [ ]:
testPath='..\\UB_keystroke_dataset\\UB_keystroke_dataset\\s1\\baseline\\001000.txt'
g = mixture.GMM(n_components=2)
g.fit(data)
np.round(g.weights_, 2)

In [ ]:
def TrainModel(path):
    """Get path of data files and train a GMM model"""
    files=ReadListOfTextFilesFromDirectoryWalk(path)
    # I am training on first 10 users
    dataFileFormat=CreateListOfTuplesFromFile(files[0:10])
    df=CreateDataFrame(dataFileFormat,columns=['Key', 'EventType','Time','User'])
    alphabetsDF=ParseAlphabetsKeys(df)
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=temp)
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
    usersFV=pd.DataFrame.from_dict(FeatureVectors)
    data= GetGMMFormatDataFromDataFrame(usersFV)
    g = mixture.GMM(n_components=2)
    g.fit(data)
    return g
    


In [ ]:
GMM= TrainModel(path)

In [ ]:
GMM.weights_

In [ ]:
def TestModel(path)

In [ ]:
if "abc" in dic:
    print("exist")